In [1]:
import os
import urllib.request
import json
import pymysql.cursors
import pandas as pd
from pymongo import MongoClient
from itertools import chain
from riotwatcher import RiotWatcher
from tqdm import tqdm
from requests.exceptions import HTTPError
from converters.data2files import get_runes_reforged_json
from converters.data2frames import game_to_dataframe as g2df
from sqlalchemy import create_engine
from datetime import datetime as dt, timedelta
import pickle
from config.constants import LEAGUES_DATA_DICT, SQL_LEAGUES_CONN, MONGODB_CREDENTIALS, \
    API_KEY, SOLOQ, REGIONS, CUSTOM_PARTICIPANT_COLS, STANDARD_POSITIONS, SCRIMS_POSITIONS_COLS, \
    TOURNAMENT_GAME_ENDPOINT, SQL_LEAGUES_ENGINE, EXCEL_EXPORT_PATH_MERGED, EXPORTS_DIR, EXCEL_EXPORT_PATH, \
    RIFT_GAMES_QUEUES, SLO, TOURNAMENT_TL_ENDPOINT, SQL_EXPORTS_ENGINE, API_KEY

In [11]:
rw = RiotWatcher(API_KEY)

In [73]:
m = rw.match.by_id(match_id=3671366054, region='EUW1')

In [74]:
tl = rw.match.timeline_by_match(match_id=3671366054, region='EUW1')

In [75]:
def timeline_participant_stats_to_dataframe(timeline):
    tl_frames = timeline['frames']
    tl_participants = [frame['participantFrames'] for frame in tl_frames]
    tl_ps_df = pd.concat(
        [pd.DataFrame(stats, index=(i,)) for i, p in enumerate(tl_participants) for p_id, stats in p.items()])
    return tl_ps_df.reset_index().rename(columns={'index': 'frame'})

In [76]:
stats = timeline_participant_stats_to_dataframe(tl)
ps = [stats.loc[stats.participantId == p_id] for p_id in range(1, 11)]

In [77]:
    def timeto_stats_from_participant(p):
        l4k = list(p.loc[p.totalGold >= 4000].head(1).frame)
        l7k = list(p.loc[p.totalGold >= 7000].head(1).frame)
        l50cs = list(p.loc[p.minionsKilled >= 50].head(1).frame)
        l100cs = list(p.loc[p.minionsKilled >= 100].head(1).frame)
        l50jcs = list(p.loc[p.jungleMinionsKilled >= 50].head(1).frame)
        l100jcs = list(p.loc[p.jungleMinionsKilled >= 100].head(1).frame)
        l50ccs = list(p.loc[p.minionsKilled + p.jungleMinionsKilled >= 50].head(1).frame)
        l100ccs = list(p.loc[p.minionsKilled + p.jungleMinionsKilled >= 100].head(1).frame)
        l6lvl = list(p.loc[p.level >= 6].head(1).frame)
        l11lvl = list(p.loc[p.level >= 11].head(1).frame)
        f5 = p.loc[p.frame == 5]
        f10 = p.loc[p.frame == 10]
        f15 = p.loc[p.frame == 15]
        f20 = p.loc[p.frame == 20]
        g5 = list(f5.totalGold)
        g10 = list(f10.totalGold)
        g15 = list(f15.totalGold)
        g20 = list(f20.totalGold)
        ccs5 = list(f5.minionsKilled + f5.jungleMinionsKilled)
        ccs10 = list(f10.minionsKilled + f10.jungleMinionsKilled)
        ccs15 = list(f15.minionsKilled + f15.jungleMinionsKilled)
        ccs20 = list(f20.minionsKilled + f20.jungleMinionsKilled)
        return {'tt4kgold': l4k[0] if l4k else None, 'tt7kgold': l7k[0] if l7k else None,
                'tt50cs': l50cs[0] if l50cs else None, 'tt100cs': l100cs[0] if l100cs else None,
                'tt50jcs': l50jcs[0] if l50jcs else None, 'tt100jcs': l100jcs[0] if l100jcs else None,
                'tt50ccs': l50ccs[0] if l50ccs else None, 'tt100ccs': l100ccs[0] if l100ccs else None,
                'ttlvl6': l6lvl[0] if l6lvl else None, 'ttlvl11': l11lvl[0] if l11lvl else None,
                'gold_at_5': g5[0] if g5 else None, 'gold_at_10': g10[0] if g10 else None,
                'gold_at_15': g15[0] if g15 else None, 'gold_at_20': g20[0] if g20 else None,
                'ccs_at_5': ccs5[0] if ccs5 else None, 'ccs_at_10': ccs10[0] if ccs10 else None,
                'ccs_at_15': ccs15[0] if ccs15 else None, 'ccs_at_20': ccs20[0] if ccs20 else None}

In [78]:
df_result = pd.concat([pd.DataFrame(timeto_stats_from_participant(p), index=(i,)) for i, p in enumerate(ps)])

In [80]:
events = list(chain.from_iterable([f['events'] for f in tl['frames']]))

In [81]:
placed_ward_events = [event for event in events if event['type'] == 'WARD_PLACED']
killed_ward_events = [event for event in events if event['type'] == 'WARD_KILL']

In [82]:
if placed_ward_events:
    print('yes')
else:
    print('no')        

yes


In [83]:
[(event['killerId'], event['wardType'], event['timestamp']) for event in killed_ward_events]

[(2, 'CONTROL_WARD', 465154),
 (6, 'CONTROL_WARD', 530944),
 (6, 'CONTROL_WARD', 655261),
 (9, 'CONTROL_WARD', 752917),
 (10, 'YELLOW_TRINKET', 842429),
 (9, 'CONTROL_WARD', 866842),
 (6, 'CONTROL_WARD', 873315),
 (1, 'BLUE_TRINKET', 892628),
 (2, 'CONTROL_WARD', 892727),
 (5, 'CONTROL_WARD', 949175),
 (6, 'YELLOW_TRINKET', 976745),
 (9, 'SIGHT_WARD', 978451),
 (4, 'CONTROL_WARD', 1015530),
 (2, 'CONTROL_WARD', 1037002),
 (3, 'SIGHT_WARD', 1246385),
 (9, 'BLUE_TRINKET', 1270484),
 (6, 'YELLOW_TRINKET', 1287473),
 (10, 'CONTROL_WARD', 1345817),
 (3, 'YELLOW_TRINKET', 1463915)]

In [84]:
df_placed = pd.DataFrame([(event['creatorId'], event['wardType'], event['timestamp']) for event in placed_ward_events]).groupby([0, 1], as_index=False).count()

In [85]:
df_placed.rename(columns={0: 'participant_id', 1: 'ward_type', 2: 'count'}, inplace=True)
df_killed.rename(columns={0: 'participant_id', 1: 'ward_type', 2: 'count'}, inplace=True)

In [86]:
df_placed.set_index('participant_id', inplace=True)
df_killed.set_index('participant_id', inplace=True)

KeyError: 'participant_id'

In [87]:
yt_p, cw_p, un_p, sw_p = df_placed.loc[df_placed['ward_type'] == 'YELLOW_TRINKET'], df_placed.loc[df_placed['ward_type'] == 'CONTROL_WARD'], df_placed.loc[df_placed['ward_type'] == 'UNDEFINED'], df_placed.loc[df_placed['ward_type'] == 'YELLOW_TRINKET']
yt_k, cw_k, un_k, sw_k = df_killed.loc[df_killed['ward_type'] == 'YELLOW_TRINKET'], df_killed.loc[df_killed['ward_type'] == 'CONTROL_WARD'], df_killed.loc[df_killed['ward_type'] == 'UNDEFINED'], df_killed.loc[df_killed['ward_type'] == 'YELLOW_TRINKET']

In [102]:
df1 = pd.concat([yt_p, cw_p, un_p, sw_p], axis=1).drop('ward_type', axis=1)
df2 = pd.concat([yt_k, cw_k, un_k, sw_k], axis=1).drop('ward_type', axis=1)

In [103]:
ward_cols = ['yellow_trinkets', 'control_wards', 'undefined', 'sight_wards']
placed_cols = [col + '_placed' for col in ward_cols]
killed_cols = [col + '_killed' for col in ward_cols]

In [104]:
df1.columns = placed_cols
df2.columns = killed_cols

In [105]:
df1.reset_index(inplace=True)
df2.reset_index(inplace=True)

In [108]:
df1 = df1.loc[df1.participant_id != 0]
df2 = df2.loc[df2.participant_id != 0]

In [109]:
df1.participant_id = df1.participant_id - 1
df2.participant_id = df2.participant_id - 1

C:\Users\david\Anaconda3\lib\site-packages\pandas\core\generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [110]:
df1.set_index('participant_id', inplace=True)
df2.set_index('participant_id', inplace=True)

In [111]:
df_result = pd.concat([df1, df2], axis=1).fillna(0)

In [2]:
import pickle

In [3]:
with open('data.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    data = pickle.load(f)

In [12]:
with open('data.pickle2', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    data2 = pickle.load(f)

In [9]:
with open('data.pickle3', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    data3 = pickle.load(f)